In [2]:
import pandas as pd


pd.set_option('display.max_columns', None)
data = pd.read_csv("../data/ai_research_papers.csv")
data.at[222653, 'author_names'] = 'Dudík, Miroslav;Haghtalab, Nika;Luo, Haipeng;Schapire, Robert E.;Syrgkanis, Vasilis;Vaughan, Jennifer Wortman'
data_exploded = data.assign(author_afids=data['author_afids'].str.split(';'), author_ids=data['author_ids'].str.split(';'), author_names = data['author_names'].str.split(';'))
data_exploded = data_exploded.explode(['author_afids', 'author_names', 'author_ids'])
data_exploded['author_rank'] = data_exploded.groupby('eid').cumcount() + 1
data_exploded = data_exploded[['eid', 'year', 'origin_ref','author_count', 'author_names', 'author_ids', 'author_afids', 'author_rank', 'citedby_count']]

aff_ids_1 = pd.read_csv("../data/ai_papers_affiliations.csv")
aff_ids_2 = pd.read_csv("../data/ai_papers_affiliations_append.csv")
aff_ids_2 = aff_ids_2[['afid', 'affiliation_name', 'org_type', 'city', 'state', 'country']].drop_duplicates()
aff_ids = pd.concat([aff_ids_1, aff_ids_2])
aff_ids = aff_ids.loc[aff_ids['country'].notna()].drop_duplicates(subset = 'afid')

data_fin = pd.merge(data_exploded, aff_ids, how = 'inner', left_on = 'author_afids', right_on = 'afid')
data_fin = data_fin.sort_values(["year","eid", "author_rank"])



/home/pkmon1d/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2,19,21,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data_dict = {}
for ind, row in data_fin.iterrows() :
    if row['eid'] not in data_dict :
        data_dict[row['eid']] = {}
        data_dict[row['eid']]['year'] = row['year'] 
        data_dict[row['eid']]['origin_ref'] = row['origin_ref'] 
        data_dict[row['eid']]['author_count'] = row['author_count'] 
        data_dict[row['eid']]['citedby_count'] = row['citedby_count'] 
        data_dict[row['eid']]['aff_country'] = []
    data_dict[row['eid']]['aff_country'].append(row['country'])


TypeError: 'list' object cannot be interpreted as an integer

In [21]:

result_dict = {}
for eid in data_dict :
    cur_eid = data_dict[eid]
    cur_affs = cur_eid['aff_country']
    processed = []
    for i in range(len(cur_affs)) :
        t_cur = cur_affs[:]
        t_cur = [item for item in t_cur if item != cur_affs[i]]
        t_cur = set(t_cur)

        if cur_affs[i] not in result_dict :
            result_dict[cur_affs[i]] = {}
            result_dict[cur_affs[i]]["total_paper"] = 0

        if cur_affs[i] not in processed :
            result_dict[cur_affs[i]]["total_paper"] += 1

        for j in t_cur :
            if j not in result_dict[cur_affs[i]] :
                result_dict[cur_affs[i]][j] = 0
            result_dict[cur_affs[i]][j] += 1
        processed.append(cur_affs[i])
        

result_dict_cite = {}
for eid in data_dict :
    cur_eid = data_dict[eid]
    cur_affs = cur_eid['aff_country']
    cur_citation = cur_eid['citedby_count']
    processed = []
    for i in range(len(cur_affs)) :
        t_cur = cur_affs[:]
        t_cur = [item for item in t_cur if item != cur_affs[i]]
        t_cur = set(t_cur)

        if cur_affs[i] not in result_dict_cite :
            result_dict_cite[cur_affs[i]] = {}
            result_dict_cite[cur_affs[i]]["total_citation"] = 0

        if cur_affs[i] not in processed :
            result_dict_cite[cur_affs[i]]["total_citation"] += cur_citation

        for j in t_cur :
            if j not in result_dict_cite[cur_affs[i]] :
                result_dict_cite[cur_affs[i]][j] = 0
            result_dict_cite[cur_affs[i]][j] += cur_citation
        processed.append(cur_affs[i])
        

In [18]:
import operator


dict(sorted(result_dict['United Kingdom'].items(), key=operator.itemgetter(1), reverse=True))

{'total_paper': 20159,
 'United States': 7553,
 'Germany': 2447,
 'China': 2432,
 'France': 1492,
 'Canada': 1394,
 'Italy': 1117,
 'Australia': 1089,
 'Netherlands': 1004,
 'Switzerland': 878,
 'Spain': 718,
 'Sweden': 716,
 'Denmark': 609,
 'Japan': 607,
 'Ireland': 567,
 'Austria': 549,
 'Hong Kong': 502,
 'Finland': 432,
 'Israel': 429,
 'Singapore': 426,
 'India': 422,
 'Belgium': 356,
 'South Korea': 334,
 'Poland': 298,
 'Greece': 283,
 'Czech Republic': 280,
 'Brazil': 254,
 'Portugal': 207,
 'Norway': 174,
 'Saudi Arabia': 170,
 'Russian Federation': 164,
 'New Zealand': 157,
 'Slovenia': 127,
 'Turkey': 114,
 'United Arab Emirates': 109,
 'Chile': 98,
 'South Africa': 80,
 'Iran': 75,
 'Luxembourg': 69,
 'Pakistan': 65,
 'Taiwan': 64,
 'Malaysia': 61,
 'Argentina': 61,
 'Egypt': 50,
 'Cyprus': 46,
 'Hungary': 46,
 'Qatar': 40,
 'Mexico': 34,
 'Estonia': 33,
 'Thailand': 31,
 'Ukraine': 27,
 'Kenya': 26,
 'Bangladesh': 23,
 'Ghana': 23,
 'Algeria': 17,
 'Malta': 17,
 'Uganda':

In [27]:
dict(sorted(result_dict_cite['China'].items(), key=operator.itemgetter(1), reverse=True))

{'total_citation': 1705119,
 'United States': 1528215,
 'Hong Kong': 582623,
 'Australia': 316519,
 'Singapore': 303432,
 'United Kingdom': 238785,
 'Canada': 154034,
 'Japan': 71902,
 'South Korea': 71671,
 'Switzerland': 59420,
 'Germany': 58734,
 'Italy': 57428,
 'India': 50921,
 'United Arab Emirates': 48806,
 'France': 46706,
 'Sweden': 44550,
 'Austria': 41493,
 'Spain': 41382,
 'Czech Republic': 39772,
 'Slovenia': 39594,
 'Turkey': 34503,
 'Iran': 34422,
 'Netherlands': 31437,
 'Finland': 31186,
 'Saudi Arabia': 28787,
 'Taiwan': 27807,
 'Israel': 22262,
 'Ukraine': 20966,
 'Macao': 20205,
 'Brazil': 15603,
 'Kazakhstan': 12551,
 'Denmark': 7407,
 'Qatar': 5626,
 'Belgium': 4216,
 'Malaysia': 3821,
 'New Zealand': 3322,
 'Ireland': 2971,
 'Egypt': 2737,
 'Norway': 2263,
 'Greece': 1858,
 'Luxembourg': 1270,
 'Iraq': 795,
 'Cyprus': 520,
 'Puerto Rico': 487,
 'Kuwait': 391,
 'Poland': 377,
 'Argentina': 288,
 'Russian Federation': 279,
 'Portugal': 254,
 'Thailand': 188,
 'Pakis

In [42]:
df_encoded = pd.get_dummies(data_fin[['eid', 'country']], columns=['country'])
df_encoded = df_encoded.groupby('eid').max().reset_index()

In [47]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(df_encoded.drop(columns='eid'), min_support=0.01, use_colnames=True)

# Generate the association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)

# Display the rules
rules

/home/pkmon1d/miniconda3/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(country_United States),(country_Canada),0.456307,0.053866,0.014808,0.032453,0.602469,-0.009771,0.977868,-0.548252
1,(country_Canada),(country_United States),0.053866,0.456307,0.014808,0.274911,0.602469,-0.009771,0.749829,-0.410865
2,(country_Hong Kong),(country_China),0.030162,0.175963,0.014879,0.493302,2.803450,0.009572,1.626290,0.663303
3,(country_China),(country_Hong Kong),0.175963,0.030162,0.014879,0.084557,2.803450,0.009572,1.059419,0.780664
4,(country_China),(country_United States),0.175963,0.456307,0.036200,0.205726,0.450849,-0.044093,0.684516,-0.596470
5,(country_United States),(country_China),0.456307,0.175963,0.036200,0.079333,0.450849,-0.044093,0.895044,-0.691387
6,(country_Germany),(country_United States),0.073905,0.456307,0.012277,0.166115,0.364042,-0.021447,0.652000,-0.653541
7,(country_United States),(country_Germany),0.456307,0.073905,0.012277,0.026905,0.364042,-0.021447,0.951700,-0.762644
8,(country_Israel),(country_United States),0.026618,0.456307,0.010086,0.378917,0.830399,-0.002060,0.875395,-0.173435
9,(country_United States),(country_Israel),0.456307,0.026618,0.010086,0.022104,0.830399,-0.002060,0.995383,-0.273073


In [17]:
0.075647 * 270000

20424.690000000002